In [1]:
import os; os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
from tensorflow import keras

In [3]:
IMG_SIZE = (224, 224)

train_ds = keras.utils.image_dataset_from_directory(
    'data/cats_dogs/',
    batch_size=16,
    label_mode='binary',
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='training')

val_ds = keras.utils.image_dataset_from_directory(
    'data/cats_dogs/',
    batch_size=16,
    label_mode='binary',
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='validation')

Found 25000 files belonging to 2 classes.
Using 17500 files for training.
Found 25000 files belonging to 2 classes.
Using 7500 files for validation.


Per prima cosa, proviamo ad utilizzare la rete in **transfer learning**. Per farlo:

* selezioniamo una rete (in questo caso, `MobileNetV3Small`) dal package `applications` di Keras;
* specifichiamo il parametro `weights` ad `imagenet`, in modo da caricare i pesi della rete già addestrata su ImageNet;
* rimuoviamo il layer di classificazione di ImageNet impostando `include_top` a `False`;
* impostiamo l'attributo `trainable` a `False` per evitare di modificare i pesi della rete;
* costruiamo un nuovo modello.

In [6]:
base_model = keras.applications.MobileNetV3Small(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False
)

base_model.trainable = False

model = keras.Sequential()
model.add(keras.layers.Lambda(keras.applications.mobilenet_v3.preprocess_input, input_shape=(224, 224, 3)))
model.add(base_model)
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1))

model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=[
        keras.metrics.BinaryAccuracy(),
        # keras.metrics.Precision(),
        # keras.metrics.Recall()
    ]
)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='checkpoints/checkpoints',
        save_weights_only=True,
        monitor='val_binary_accuracy',
        save_best_only=True),
    keras.callbacks.EarlyStopping(
        monitor='val_binary_accuracy',
        min_delta=0.1,
        patience=3,
        restore_best_weights=True),
    keras.callbacks.TensorBoard(log_dir='logs')
]

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks)

Epoch 1/20
1094/1094 [==============================] - ETA: 0s - loss: 0.4218 - binary_accuracy: 0.9519

KeyboardInterrupt: 

Proviamo adesso ad effettuare il **fine tuning**. Per farlo, impostiamo `trainable` a `True`, e compiliamo il modello usando un learning rate molto basso (in questo caso, 0.00001).

In [ ]:
base_model.trainable = True

model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(1e-5),
    metrics=[
        keras.metrics.BinaryAccuracy(),
        keras.metrics.Precision(),
        keras.metrics.Recall()
    ]
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
    callbacks=callbacks)